# CNN + Machine Learning Classification for Keypoint Dataset

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
# Load และเตรียมข้อมูล
df = pd.read_csv(r"D:\git_project\workshop_lifting\app\Train\Labeled_Dataset__English_.csv")  # หรือใช้ DataFrame ที่มีอยู่

X = df.drop(columns=["time", "label"]).values
y = df["label"].values

In [ ]:
X = df.drop(columns=["time", "label"]).values
y = df["label"].values

In [ ]:
# สร้าง window ขนาด 30 frame (เช่น 1 วิ)
window_size = 30
stride = 15

X_windows, y_windows = [], []

for start in range(0, len(X) - window_size, stride):
    end = start + window_size
    if len(set(y[start:end])) == 1:  # เอาเฉพาะช่วงที่ label เดียวกัน
        X_windows.append(X[start:end])
        y_windows.append(y[start])

X_windows = np.array(X_windows)  # shape: [N, 30, features]
y_windows = np.array(y_windows)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_windows)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_windows, y_encoded, test_size=0.2, random_state=42)

# PyTorch Dataset
class KeypointDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = KeypointDataset(X_train, y_train)
test_dataset = KeypointDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [8]:

import torch.nn as nn
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, num_features, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 6, 128)  # ปรับตาม input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [B, features, time]
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

model = CNNClassifier(num_features=X.shape[1], num_classes=len(label_encoder.classes_))


In [9]:

import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(300):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()


In [10]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_rf = X_windows.reshape(X_windows.shape[0], -1)  # flatten
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_encoded, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

print(classification_report(y_test_rf, y_pred_rf, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

       empty       0.71      0.83      0.77        12
        hold       0.85      0.69      0.76        16
        lift       0.00      0.00      0.00         1
       place       0.00      0.00      0.00         2
     unknown       1.00      1.00      1.00         2

    accuracy                           0.70        33
   macro avg       0.51      0.50      0.51        33
weighted avg       0.73      0.70      0.71        33



d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_rf = X_windows.reshape(X_windows.shape[0], -1)  # flatten
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_encoded, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

print(classification_report(y_test_rf, y_pred_rf, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

       empty       0.62      0.83      0.71        12
        hold       0.83      0.62      0.71        16
        lift       0.00      0.00      0.00         1
       place       0.00      0.00      0.00         2
     unknown       1.00      1.00      1.00         2

    accuracy                           0.67        33
   macro avg       0.49      0.49      0.49        33
weighted avg       0.69      0.67      0.67        33



d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programs\Anaconda\envs\workshop_lifting\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",